In [1]:
import sys
import importlib
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import sys
import awkward as ak
import scipy
import ROOT
sys.path.insert(0,"../")


Welcome to JupyROOT 6.30/07


In [2]:
sys.path.insert(0,"../python/HNL_Plotting_HelperFunctions")
import MuonSystemReader
import ABCD_e
import Processing_Helpers
import analysis_helpers


/cvmfs/cms.cern.ch/el9_amd64_gcc12/lcg/root/6.30.07-024df6516c17fd2edef848a927a788f1/lib/ROOT/_facade.py:154: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  return _orig_ihook(name, *args, **kwds)
/cvmfs/cms.cern.ch/el8_amd64_gcc12/external/py3-numpy/1.24.3-0831476b9e4ddf0b4b9eb1f4c971c0d1/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, w

In [3]:
HNL_2GeV_10ctau_path = "root://cmseos.fnal.gov//store/group/lpclonglived/amalbert/HNL_Tau_Search/analyzer_update_111825/HNL_e_mN_2_ctau_1000/normalized/HNL_e_mN_2_ctau_1000.root"
signal_events  = MuonSystemReader.loadTree_nanoFactory(HNL_2GeV_10ctau_path)

/uscms/home/amalbert/.local/lib/python3.9/site-packages/dask_awkward/lib/structure.py:911: UserWarning: Please ensure that dask.awkward<nCscRechitClusters, npartitions=1>
        is partitionwise-compatible with dask.awkward<getitem, npartitions=1>
        (e.g. counts comes from a dak.num(array, axis=1)),
        otherwise this unflatten operation will fail when computed!
  warnings.warn(


In [5]:
signal_xSec = 1
processed_lumi = 109080

In [6]:
generated_signalEvents = ak.count(signal_events.evtNum)
signal_normalization_factor = (processed_lumi*signal_xSec/generated_signalEvents).compute()
print(signal_normalization_factor)

0.24678733031674208


In [7]:
sizeCut=160
phiCut=2

### Check contamination at 10 pb (rough UL from Run 2 search), no additional DNN/StationVeto Selections

In [9]:
clusterSize_allCutsDNN0, dPhi_allCutsDNN0 = ABCD_e.return_clusterSize_dPhiClusterE_allSelectionsDNN(signal_events, passID = 'lepLooseId', failID = 'lepMediumId', blind=False, DNN_cut=0)
clusterSize_allCutsDNN0MT, dPhi_allCutsDNN0MT = ABCD_e.return_clusterSize_dPhiClusterE_allSelectionsDNN(signal_events, passID = 'lepMediumId', failID = 'lepTightId', blind=False, DNN_cut=0)
clusterSize_allCutsDNN0SR, dPhi_allCutsDNN0SR = ABCD_e.return_clusterSize_dPhiClusterE_allSelectionsDNN(signal_events, passID = 'lepTightId', failID = None, blind=False, DNN_cut=0)

/uscms/home/amalbert/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/cvmfs/cms.cern.ch/el8_amd64_gcc12/external/py3-numpy/1.24.3-0831476b9e4ddf0b4b9eb1f4c971c0d1/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])

In [10]:
print(f"signal contamination VR1 no DNN/StatonVeto {ak.count_nonzero(clusterSize_allCutsDNN0)*signal_normalization_factor}")
print(f"signal contamination VR2 no DNN/StatonVeto {ak.count_nonzero(clusterSize_allCutsDNN0MT)*signal_normalization_factor}")
print(f"signal in SR {ak.count_nonzero(clusterSize_allCutsDNN0SR)*signal_normalization_factor}")

print("in bin A")
print(f"bin A VR1: {ak.count_nonzero((clusterSize_allCutsDNN0>160) & (dPhi_allCutsDNN0>2))*signal_normalization_factor}")
print(f"bin A VR2: {ak.count_nonzero((clusterSize_allCutsDNN0MT>160) & (dPhi_allCutsDNN0MT>2))*signal_normalization_factor}")
print(f"bin A SR: {ak.count_nonzero((clusterSize_allCutsDNN0SR>160) & (dPhi_allCutsDNN0SR>2))*signal_normalization_factor}")  

signal contamination VR1 no DNN/StatonVeto 17.028325791855202
signal contamination VR2 no DNN/StatonVeto 19.98977375565611
signal in SR 164.60714932126697
in bin A
bin A VR1: 10.118280542986426
bin A VR2: 12.339366515837105
bin A SR: 111.05429864253394


### Signal Contamination with StationVeto Applied

In [11]:
clusterSize_allCutsStationVeto, dPhi_allCutsStationVeto = ABCD_e.return_clusterSize_dPhiClusterE_allSelections(signal_events, passID = 'lepLooseId', failID = 'lepMediumId', blind=False)
clusterSize_allCutsStationVetoMT, dPhi_allCutsStationVetoMT = ABCD_e.return_clusterSize_dPhiClusterE_allSelections(signal_events, passID = 'lepMediumId', failID = 'lepTightId', blind=False)
clusterSize_allCutsStationVetoSR, dPhi_allCutsStationVetoSR = ABCD_e.return_clusterSize_dPhiClusterE_allSelections(signal_events, passID = 'lepTightId', failID = None, blind=False)

/uscms/home/amalbert/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/cvmfs/cms.cern.ch/el8_amd64_gcc12/external/py3-numpy/1.24.3-0831476b9e4ddf0b4b9eb1f4c971c0d1/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])

In [12]:
print(f"signal contamination VR1 with Station12Veto {ak.count_nonzero(clusterSize_allCutsStationVeto)*signal_normalization_factor}")
print(f"signal contamination VR2 with Station12Veto {ak.count_nonzero(clusterSize_allCutsStationVetoMT)*signal_normalization_factor}")
print(f"signal in SR with Station12Veto {ak.count_nonzero(clusterSize_allCutsStationVetoSR)*signal_normalization_factor}")

print("in bin A")
print(f"bin A VR1: {ak.count_nonzero((clusterSize_allCutsStationVeto>160) & (dPhi_allCutsStationVeto>2))*signal_normalization_factor}")
print(f"bin A VR2: {ak.count_nonzero((clusterSize_allCutsStationVetoMT>160) & (dPhi_allCutsStationVetoMT>2))*signal_normalization_factor}")
print(f"bin A SR: {ak.count_nonzero((clusterSize_allCutsStationVetoSR>160) & (dPhi_allCutsStationVetoSR>2))*signal_normalization_factor}")  

signal contamination VR1 with Station12Veto 4.935746606334842
signal contamination VR2 with Station12Veto 5.4293212669683255
signal in SR with Station12Veto 35.043800904977374
in bin A
bin A VR1: 3.948597285067873
bin A VR2: 3.455022624434389
bin A SR: 26.899819004524886


### Signal Contamination with DNN>0.99975

In [13]:
clusterSize_allCutsDNN, dPhi_allCutsDNN = ABCD_e.return_clusterSize_dPhiClusterE_allSelectionsDNN(signal_events, passID = 'lepLooseId', failID = 'lepMediumId', blind=False, DNN_cut=0.99975)
clusterSize_allCutsDNNMT, dPhi_allCutsDNNMT = ABCD_e.return_clusterSize_dPhiClusterE_allSelectionsDNN(signal_events, passID = 'lepMediumId', failID = 'lepTightId', blind=False, DNN_cut=0.99975)
clusterSize_allCutsDNNSR, dPhi_allCutsDNNSR = ABCD_e.return_clusterSize_dPhiClusterE_allSelectionsDNN(signal_events, passID = 'lepTightId', failID = None, blind=False, DNN_cut=0.99975)

/uscms/home/amalbert/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/cvmfs/cms.cern.ch/el8_amd64_gcc12/external/py3-numpy/1.24.3-0831476b9e4ddf0b4b9eb1f4c971c0d1/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])

In [14]:
print(f"signal contamination VR1 with DNN {ak.count_nonzero(clusterSize_allCutsDNN)*signal_normalization_factor}")
print(f"signal contamination VR2 with DNN {ak.count_nonzero(clusterSize_allCutsDNNMT)*signal_normalization_factor}")
print(f"signal in SR with DNN {ak.count_nonzero(clusterSize_allCutsDNNSR)*signal_normalization_factor}")

print("in bin A")
print(f"bin A VR1: {ak.count_nonzero((clusterSize_allCutsDNN>160) & (dPhi_allCutsDNN>2))*signal_normalization_factor}")
print(f"bin A VR2: {ak.count_nonzero((clusterSize_allCutsDNNMT>160) & (dPhi_allCutsDNNMT>2))*signal_normalization_factor}")
print(f"bin A SR: {ak.count_nonzero((clusterSize_allCutsDNNSR>160) & (dPhi_allCutsDNNSR>2))*signal_normalization_factor}")

signal contamination VR1 with DNN 1.2339366515837105
signal contamination VR2 with DNN 2.467873303167421
signal in SR with DNN 15.054027149321266
in bin A
bin A VR1: 0.9871493212669683
bin A VR2: 1.2339366515837105
bin A SR: 10.858642533936651
